In [0]:
# Import functions
from pyspark.sql.functions import col, current_timestamp

# Define variables used in code below
file_path = "/FileStore/stream/events/"
username = spark.sql("SELECT regexp_replace(current_user(), '[^a-zA-Z0-9]', '_')").first()[0]
table_name = f"{username}_etl_quickstart"
checkpoint_path = f"/tmp/{username}/_checkpoint/etl_quickstart"

# Clear out data from previous demo execution
spark.sql(f"DROP TABLE IF EXISTS {table_name}")
dbutils.fs.rm(checkpoint_path, True)





Out[10]: True

In [0]:
dbutils.fs.rm(f"dbfs:/user/hive/warehouse/{username}_etl_quickstart", True)


Out[11]: False

In [0]:
%fs ls /databricks-datasets/structured-streaming/events

path,name,size,modificationTime
dbfs:/databricks-datasets/structured-streaming/events/file-0.json,file-0.json,72530,1469673865000
dbfs:/databricks-datasets/structured-streaming/events/file-1.json,file-1.json,72961,1469673866000
dbfs:/databricks-datasets/structured-streaming/events/file-10.json,file-10.json,73025,1469673878000
dbfs:/databricks-datasets/structured-streaming/events/file-11.json,file-11.json,72999,1469673879000
dbfs:/databricks-datasets/structured-streaming/events/file-12.json,file-12.json,72987,1469673880000
dbfs:/databricks-datasets/structured-streaming/events/file-13.json,file-13.json,73006,1469673881000
dbfs:/databricks-datasets/structured-streaming/events/file-14.json,file-14.json,73003,1469673882000
dbfs:/databricks-datasets/structured-streaming/events/file-15.json,file-15.json,73007,1469673883000
dbfs:/databricks-datasets/structured-streaming/events/file-16.json,file-16.json,72978,1469673885000
dbfs:/databricks-datasets/structured-streaming/events/file-17.json,file-17.json,73008,1469673886000


Żeby sprawdzić czy stream działa kopiuj po jednym bądź kilku plikach 

In [0]:
dbutils.fs.cp("/databricks-datasets/structured-streaming/events/file-4.json","/FileStore/stream/events/file-4.json",True)

Out[12]: True

In [0]:
display(dbutils.fs.ls("/FileStore/stream/events/"))

path,name,size,modificationTime
dbfs:/FileStore/stream/events/file-4.json,file-4.json,72992,1746268891000


In [0]:
spark.read.format("json").load("dbfs:/FileStore/stream/events/").count()

Out[14]: 2000

Dokończyć kod autoloadera 
1. Dodaj opcje 'cloudfiles'
2. Dodaj kolumnę z metadanych 'source_file'
3. Dane zapisać do tabeli

In [0]:

from pyspark.sql.functions import input_file_name

(spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "json")
  .option("cloudFiles.schemaLocation", checkpoint_path)
  .load(file_path)
  .withColumn("source_file", input_file_name())
  .writeStream
  .option("checkpointLocation", checkpoint_path)
  .trigger(availableNow=True)
  .toTable(table_name))

df = spark.sql(f"SELECT * FROM {table_name} LIMIT 10")
df.display()

action,time,_rescued_data,source_file


In [0]:


df = spark.sql(f"select count(*) from {table_name}")
df.display()

count(1)
0


Sprawdzć metadane 

In [0]:

display(dbutils.fs.ls(f"/tmp/{username}/_checkpoint/etl_quickstart/sources/0/metadata/"))

path,name,size,modificationTime
dbfs:/tmp/wtomczyk_student_agh_edu_pl/_checkpoint/etl_quickstart/sources/0/metadata,metadata,133,1746268906000


In [0]:

display(dbutils.fs.ls(f"/tmp/{username}/_checkpoint/etl_quickstart/sources/0/metadata/"))

path,name,size,modificationTime
dbfs:/tmp/wtomczyk_student_agh_edu_pl/_checkpoint/etl_quickstart/sources/0/metadata,metadata,133,1746268906000
